In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('/content/WELFake_Dataset.csv')

In [3]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
df.isnull().sum()

,0
Unnamed: 0,0
title,558
text,39
label,0


In [5]:
df=df.dropna()

In [6]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [7]:
X=df.drop('label',axis=1)

In [8]:
y=df['label']

In [10]:
X.shape

(71537, 3)

In [11]:
y.shape

(71537,)

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.19.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
voc_size=5000

One hot encoding


In [16]:
messages=X.copy()

In [17]:

messages['title'].iloc[1]

'UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MOST CHARLOTTE RIOTERS WERE “PEACEFUL” PROTESTERS…In Her Home State Of North Carolina [VIDEO]'

In [20]:
messages

,Unnamed: 0,title,text
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...
72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'].iloc[i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [21]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [22]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[3394, 185, 2357, 46, 1747, 4266, 331, 1102, 1672, 2607, 1404, 4218],
 [995,
  4269,
  1525,
  3352,
  1224,
  3120,
  135,
  4242,
  2206,
  4213,
  2616,
  4104,
  2239,
  4218],
 [1265, 3511, 103, 476, 1216, 4331, 2425, 95, 4360, 4078, 3358, 4257],
 [2497, 4969, 4027, 2872, 4478, 4156, 3546, 3143, 2648, 676, 2174],
 [1255, 2425, 2214, 41, 3105, 396, 4891, 2780, 2214],
 [926, 4752, 1721, 2333, 1223, 3847, 3635, 2038, 3954, 105, 1239],
 [2873, 134, 4310, 2224, 4969, 452, 4331, 4255, 4255, 4218],
 [1964,
  1217,
  284,
  192,
  1551,
  1857,
  3517,
  895,
  1149,
  1964,
  3056,
  2660,
  4144,
  3354,
  4218],
 [1067, 1148, 2942, 3893, 3897, 3674, 3959, 1148],
 [1819, 2008, 3926, 1450, 3278, 3389, 578, 1604],
 [1606, 1300, 3710, 2604, 1867, 2556, 1856, 4283, 1856, 1082],
 [2063, 3657, 2224, 1327, 1742, 846, 1624, 4517, 1691],
 [1059, 402, 4737, 3657, 2613, 4369, 646, 3975, 956, 4765, 2196],
 [3994, 4334, 3225, 1727, 2157, 2003, 4175, 3823, 4145, 3721],
 [3025, 2928, 4881, 4456, 2713

In [23]:
onehot_repr[1]

[995,
 4269,
 1525,
 3352,
 1224,
 3120,
 135,
 4242,
 2206,
 4213,
 2616,
 4104,
 2239,
 4218]

embeddibg representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[3394  185 2357 ...    0    0    0]
 [ 995 4269 1525 ...    0    0    0]
 [1265 3511  103 ...    0    0    0]
 ...
 [1889 1032 2484 ...    0    0    0]
 [2224 2043 1927 ...    0    0    0]
 [4684 4333 4716 ...    0    0    0]]


In [25]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
len(embedded_docs),y.shape

(71537, (71537,))

In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

Model Training


In [29]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 9s 7ms/step - accuracy: 0.7900 - loss: 0.4079 - val_accuracy: 0.8868 - val_loss: 0.2636
Epoch 2/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9049 - loss: 0.2299 - val_accuracy: 0.8944 - val_loss: 0.2579
Epoch 3/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9230 - loss: 0.1952 - val_accuracy: 0.8952 - val_loss: 0.2623
Epoch 4/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9316 - loss: 0.1774 - val_accuracy: 0.8952 - val_loss: 0.2663
Epoch 5/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.9411 - loss: 0.1557 - val_accuracy: 0.8850 - val_loss: 0.3043
Epoch 6/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9528 - loss: 0.1311 - val_accuracy: 0.8943 - val_loss: 0.2737
Epoch 7/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.9595 - loss: 0.1137 - val_accuracy: 0.8912 - val_loss: 0.3173
Epoch 8/10
749/749 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9654 - loss: 0.0933 - val_accuracy: 0

Performance matrics


In [30]:
y_pred=model.predict(X_test)

738/738 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [31]:
y_pred=np.where(y_pred > 0.6, 1,0)

In [32]:
from sklearn.metrics import confusion_matrix

In [33]:
confusion_matrix(y_test,y_pred)

array([[10451,  1242],
       [ 1339, 10576]])

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8906726533378516

In [35]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89     11693
           1       0.89      0.89      0.89     11915

    accuracy                           0.89     23608
   macro avg       0.89      0.89      0.89     23608
weighted avg       0.89      0.89      0.89     23608

